In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
google_api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
if google_api_key=="":
    print("api key not found")
else:
    print("Api key found")

Api key found


In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
import google.generativeai as genai
from IPython.display import Markdown, display
from llama_index.core import ServiceContext, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding

h:\Chirag_DataScience_Course\April_2024\QASystem\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
genai.configure(api_key=google_api_key)

In [6]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


In [7]:
documents = SimpleDirectoryReader("../Data")

In [8]:
doc = documents.load_data()

In [9]:
doc[0].text

'Full Terms & Conditions of access and use can be found at\nhttps://www.tandfonline.com/action/journalInformation?journalCode=tpsr20\nPsychotherapy Research\nISSN: (Print) (Online) Journal homepage: https://www.tandfonline.com/loi/tpsr20\nUnderstanding the relationship between\npatient language and outcomes in internet-\nenabled cognitive behavioural therapy: A deep\nlearning approach to automatic coding of session\ntranscripts\nM. P. Ewbank, R. Cummins, V. Tablan, A. Catarino, S. Buchholz & A. D.\nBlackwell\nTo cite this article:  M. P. Ewbank, R. Cummins, V. Tablan, A. Catarino, S. Buchholz\n& A. D. Blackwell (2021) Understanding the relationship between patient language and\noutcomes in internet-enabled cognitive behavioural therapy: A deep learning approach to\nautomatic coding of session transcripts, Psychotherapy Research, 31:3, 300-312, DOI:\n10.1080/10503307.2020.1788740\nTo link to this article:  https://doi.org/10.1080/10503307.2020.1788740\n© 2020 Ieso Digital Health. Publis

In [10]:
model = Gemini(models='gemini-pro', api_key=google_api_key)


In [11]:
gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")

In [12]:
service_context = ServiceContext.from_defaults(llm=model, embed_model=gemini_embed_model, chunk_size=800, chunk_overlap=20)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [13]:
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.node_parser = SentenceSplitter(chunk_size=800, chunk_overlap=20)

In [15]:
index = VectorStoreIndex.from_documents(doc,transformations=Settings.transformations)

In [16]:
index

In [17]:
index.storage_context.persist()

In [18]:
query_engine = index.as_query_engine()

In [30]:
response = query_engine.query("please five a summary of the doc")

In [31]:
response.response

'This document discusses the use of digital mental health services and the importance of research in this field. It highlights the need for evidence-based interventions and the role of technology in providing accessible and effective mental health care. \n'

In [22]:
response1 = query_engine.query("What are the challenges of diagnosing depression?")

In [23]:
response1.response

'Current diagnostic systems may not be sufficient to explore the full range of human behavior and to describe the rich underlying complexity of mental health disorders.  The classification of symptoms and the origin of diagnoses was built on expert consensus and the agglomeration of different phenotypes under the same diagnosis. This leads to a substantial degree of heterogeneity, as more than 100 combinations of symptom criteria can lead to the same unitary diagnosis of depression. \n'